In [ ]:
!pip install torchvision

In [6]:
import numpy as np
import autokeras as ak
import pandas as pd
from keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
import os

In [8]:
# Load the CSV file containing image filenames and labels with specified column names
data_path = 'VehicleDataPreprocessedReduced.csv'
data_df = pd.read_csv(data_path, names=['filename', 'label'])

# Function to load and preprocess images from filenames and directory
def load_and_preprocess_image(image_filename, directory):
    image_path = os.path.join(directory, image_filename)
    img = load_img(image_path, target_size=(256, 256))
    img_array = img_to_array(img) / 255.0  # Normalize pixel values
    return img_array

# Directory containing the images
image_directory = 'Vehicle Classification V2.v3i.retinanet/train'

# Load images from filenames and directory and labels
x = np.array([load_and_preprocess_image(filename, image_directory) for filename in data_df['filename']])
y = np.array(data_df['label'])

# Convert labels to integers
label_to_index = {label: index for index, label in enumerate(np.unique(y))}
y = np.array([label_to_index[label] for label in y])

# Split the data into training and testing sets (80% train, 20% test)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


In [9]:
clf = ak.ImageClassifier(
    max_trials = 5,
    objective = 'val_accuracy',
    overwrite = True,
)

# Search for the best model architecture
clf.fit(x_train, y_train, epochs = 10)





Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
vanilla           |vanilla           |image_block_1/block_type
True              |True              |image_block_1/normalize
False             |False             |image_block_1/augment
3                 |3                 |image_block_1/conv_block_1/kernel_size
1                 |1                 |image_block_1/conv_block_1/num_blocks
2                 |2                 |image_block_1/conv_block_1/num_layers
True              |True              |image_block_1/conv_block_1/max_pooling
False             |False             |image_block_1/conv_block_1/separable
0.25              |0.25              |image_block_1/conv_block_1/dropout
32                |32                |image_block_1/conv_block_1/filters_0_0
64                |64                |image_block_1/conv_block_1/filters_0_1
flatten           |flatten           |classification_head_1/spatial_reduction_1/reduction_type
0.5               |0.5       

In [8]:
results = clf.evaluate(x_test, y_test)
print(results)
print("Test accuracy:", results[1])

2/2 [==============================] - 2s 631ms/step - loss: 1.1038e-08 - accuracy: 1.0000
[1.1037893266063747e-08, 1.0]
Test accuracy: 1.0


In [9]:
from sklearn.metrics import classification_report

In [10]:
y_pred = clf.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Calculate precision, recall, accuracy, and F1 score
report = classification_report(y_test, y_pred_classes)
print(report)

2/2 [==============================] - 2s 867ms/step
              precision    recall  f1-score   support

           0       0.33      1.00      0.50        18
           1       0.00      0.00      0.00        18
           2       0.00      0.00      0.00        18

    accuracy                           0.33        54
   macro avg       0.11      0.33      0.17        54
weighted avg       0.11      0.33      0.17        54



c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f

In [ ]:

clf.export_model().save("Chicken Model")